In [19]:
import torch
from torch import nn
from torch.nn import functional as F
import dlc_practical_prologue as prologue

In [35]:
#generate train and test data
#target classes: a>b->0   a<=b->1

[train_input,train_target,train_classes,
 test_input,test_target,test_classes]=prologue.generate_pair_sets(1000)

train_target = train_target.float()

In [36]:
nb_epochs = 25


# Conv -> maxpool -> relu
class ConvNet1(nn.Module):
    def __init__(self):
        super(ConvNet1, self).__init__()
        nb_hidden = 200
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3)
        self.fc1 = nn.Linear(4 * 64, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 1)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2))
        x = F.relu(self.fc1(x.view(-1, 4 * 64)))
        x = self.fc2(x)
        return x

# Conv -> relu -> maxpool
class ConvNet2(nn.Module):
    def __init__(self):
        super(ConvNet2, self).__init__()
        nb_hidden = 200
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.fc1 = nn.Linear(9 * 32, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2)
        x = F.relu(self.fc1(x.view(-1, 9 * 32)))
        x = self.fc2(x)
        return x

In [58]:
def train_model(model, train_input, train_target):
    criterion = nn.MSELoss()
    eta = 1e-1

    for e in range(25):
        sum_loss = 0
        for b in range(0, train_input.size(0)):
            output = model(train_input[b])
            loss = criterion(output, train_target[b])
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.item()
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print(e, sum_loss)
        
        
def compute_nb_errors(model, train_input, target):
    nb_errors = 0

    for b in range(0, train_input.size(0)):
        output = model(train_input[b])
        _, predicted_classes = output.data.max(1)
        for k in range(mini_batch_size):
            if target.data[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

In [59]:
model = ConvNet1()
train_model(model, train_input, train_target)
nb_test_errors = compute_nb_errors(model, test_input, test_target)
print('test error Net2 {:0.2f}%% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                   nb_test_errors, test_input.size(0)))

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 2, 3, 3], but got 3-dimensional input of size [2, 14, 14] instead